In [1]:
import tensorflow as tf
import numpy as np
sess = tf.InteractiveSession()

In [2]:
# convenience function for clearing graph and starting a new session
def reset():
    global sess
    sess.close()
    tf.reset_default_graph()
    sess = tf.InteractiveSession()
def ops():
    return tf.get_default_graph().get_operations()
def tensors():
    return tf.get_default_graph().get_tensors()


In [3]:
nA = np.full([3,5],2.0)
nB = np.full([5,2],7.0)
# nC = nA * nB   # shape error
# nC = nA + nB   # shape error
nC = np.matmul(nA,nB)
print("numpy A: shape =", np.shape(nA), "dtype =", nA.dtype.name)
print(nA)
print("numpy B: shape =", np.shape(nB), "dtype =", nB.dtype.name)
print(nB)
print("numpy C: shape =", np.shape(nC), "dtype =", nC.dtype.name)
print(nC)


numpy A: shape = (3, 5) dtype = float64
[[ 2.  2.  2.  2.  2.]
 [ 2.  2.  2.  2.  2.]
 [ 2.  2.  2.  2.  2.]]
numpy B: shape = (5, 2) dtype = float64
[[ 7.  7.]
 [ 7.  7.]
 [ 7.  7.]
 [ 7.  7.]
 [ 7.  7.]]
numpy C: shape = (3, 2) dtype = float64
[[ 70.  70.]
 [ 70.  70.]
 [ 70.  70.]]


In [4]:
A = tf.Variable(nA)
B = tf.Variable(nB)  
# C = A + B    # shape error
# C = tf.add(A,B)  # same as C=A+B
# C = A * B    # shape error
# C = tf.multiply(A,B) # same as C=A*B
preC = tf.matmul(A,B)
# C = tf.Variable(tf.matmul(A, B))   # no overloaded operator for matmul
C = tf.Variable(preC)
print("A: shape =", A.shape, "optype =", type(A), "dtype =", A.dtype.name)
print("B: shape =", B.shape, "optype =", type(B), "dtype =", B.dtype.name)
print("preC: shape =", preC.shape, "optype =", type(preC), "dtype =", preC.dtype.name)
print("C: shape =", C.shape, "optype =", type(C), "dtype =", C.dtype.name)

A: shape = (3, 5) optype = <class 'tensorflow.python.ops.variables.Variable'> dtype = float64_ref
B: shape = (5, 2) optype = <class 'tensorflow.python.ops.variables.Variable'> dtype = float64_ref
preC: shape = (3, 2) optype = <class 'tensorflow.python.framework.ops.Tensor'> dtype = float64
C: shape = (3, 2) optype = <class 'tensorflow.python.ops.variables.Variable'> dtype = float64_ref


In [5]:
tf.global_variables_initializer().run()
C.eval()


array([[ 70.,  70.],
       [ 70.,  70.],
       [ 70.,  70.]])

In [6]:
x = C * 0.5
print(x.eval())
y = C * 0.2
print(y.eval())


[[ 35.  35.]
 [ 35.  35.]
 [ 35.  35.]]
[[ 14.  14.]
 [ 14.  14.]
 [ 14.  14.]]


In [7]:
reset()
C = tf.Variable (0.45 + 0.2j)

Z = tf.Variable(C)
W = tf.Variable(C)
S = tf.Variable(3.0+0j, name="S")
T = tf.Variable(3.0+0j, name="T")
R = tf.Variable(3.0+0j, name="R")
M = tf.Variable(0.0, name="M")
#P = tf.placeholder(tf.complex128, shape=(), name="P")
#R = tf.Variable(P)

a = tf.placeholder(tf.float32, shape=())
b = tf.Variable(a)
c = a * a
d = c + 2.0
e = d + b

# new* are NOT Variables,
# but rather "add" Tensors (with "multiply" Tensor intermediaries)
newZ = Z*Z + C
newW = Z*Z + C
newS = Z*Z + C
newT = Z*Z + C

#tf.global_variables_initializer().run(feed_dict={P: 2.0+0.5j})
#print(R.eval())
# tf.global_variables_initializer().run(feed_dict={P: 7.0})
# sess.run(R, feed_dict={P: 9.0})
# print(sess.run(R, feed_dict={P: 11.0+0.5j}))
# R.eval()

tf.global_variables_initializer().run(feed_dict={a: 7.0})
print(d.eval(feed_dict={a: 3.0}))
print(d.eval(feed_dict={a: 4.0}))
#print(b.eval(feed_dict={a: 1.0}))
#S.eval(feed_dict={a:20})
#print(sess.run(S, feed_dict={a:20}))
b2 = b.assign(e)
print(b2.eval(feed_dict={a: 1.3}))
print(b.eval())
b.assign(e).eval(feed_dict={a: 1.5})
print("b", b.eval())

11.0
18.0
10.69
10.69
b 14.94


In [8]:
# simple direct loop
for i in range(5):
    R = R + C
    print(R.eval())


(3.45+0.2j)
(3.9+0.4j)
(4.35+0.6j)
(4.8+0.8j)
(5.25+1j)


In [9]:
# direct loop
# if redoing loop repeat
maxiter = 10
for i in range(maxiter):
    # why doesn't assign work?
    # Z.assign(newZ)   
    # using direct tensor ops instead
    Z = Z*Z + C
    # Have we diverged with this new value?
    not_diverged = (tf.abs(Z) <= 4)
    # if not, increment iterations count for the point
    M = M + tf.cast(not_diverged, tf.float32)
    # don't need to call run() on each iterations, 
    print(M.eval(), Z.eval())
    
# M.eval and Z.eval anytime after loop will run entire unrolled loop
print("postloop ===>", M.eval(), Z.eval())

cass = C.assign_add(1.0+0.0)
#btf.global_variables_initializer().run()
sess.run(cass)
#sess.run(Z)
print("postloop ===>", M.eval(), Z.eval())

print(cass)
print(cass.eval())
print(C.eval())
sess.run(Z)
C.assign_add(-3.0+0.1j).eval()
sess.run(Z)



1.0 (0.6125+0.38j)
2.0 (0.68075625+0.6655j)
3.0 (0.470538821914+1.10608656875j)
4.0 (-0.552020714641+1.24091334199j)
5.0 (-0.785139052934-1.1700197397j)
6.0 (-0.302502858856+2.03725638069j)
7.0 (-3.60890558105-1.03255175876j)
7.0 (12.4080363584+7.65276360986j)
7.0 (95.8445754038+190.111538227j)
7.0 (-26955.7643328+36442.5193215j)
postloop ===> 7.0 (-26955.7643328+36442.5193215j)
postloop ===> 1.0 (-1.48438775067e+162+1.79674832954e+162j)
Tensor("AssignAdd:0", shape=(), dtype=complex128_ref)
(2.45+0.2j)
(2.45+0.2j)


(-0.34434033271740883+0.26781405797657259j)

In [10]:
# group and assign inline constructed Op/Tensor
for i in range(10):
    stepW = tf.group(
        W.assign(W*W+C)
    )
    # works only if call run() on each iteration
    stepW.run()
    print(W.eval())
    
#stepW.run()
#W.eval()

(-0.3875+0.48j)
(-0.63024375-0.072j)
(-0.157976815586+0.3907551j)
(-0.677732873913+0.176539507256j)
(-0.121844349239+0.0607067447762j)
(-0.53883926342+0.285206452377j)
(-0.340994968675-0.00736086944234j)
(-0.433776613738+0.30502003889j)
(-0.454875073499+0.0353788808165j)
(-0.344340332717+0.267814057977j)


In [11]:
# group and assign to prior Op/Tensor
for i in range(5):
    stepS = tf.group(
        S.assign(newS)
    )
    stepS.run()
    print(S.eval())
stepS.run()
S.eval()

(-0.3875+0.48j)
(-0.3875+0.48j)
(-0.3875+0.48j)
(-0.3875+0.48j)
(-0.3875+0.48j)


(-0.38749999999999984+0.48000000000000009j)

In [12]:
v = tf.get_variable("v", shape=(), initializer=tf.zeros_initializer())
assignment = v.assign_add(1)
tf.global_variables_initializer().run()
assignment.run()


InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder' with dtype float
	 [[Node: Placeholder = Placeholder[dtype=DT_FLOAT, shape=[], _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

Caused by op 'Placeholder', defined at:
  File "/home/user1/anaconda3/envs/tflow14_py35/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/user1/anaconda3/envs/tflow14_py35/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/user1/anaconda3/envs/tflow14_py35/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/user1/anaconda3/envs/tflow14_py35/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/user1/anaconda3/envs/tflow14_py35/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/home/user1/anaconda

In [ ]:
x = tf.Variable(0)
y = tf.assign(x, 2)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(x.eval())
    print(y.eval())
    print(x.eval())

In [ ]:
x = tf.placeholder(dtype=tf.float32, shape=())
x2 = x * x
y = x2 + 3.0

with tf.Session() as session:
    # session.run(y) # will fail!
    # session.run(y, feed_dict={x: [[1, 2], [3, 4]]}) # OK
    print(y.eval(feed_dict={x: 2.0}))
    
    print(y.eval(feed_dict={x: 3}))
    print(y.eval(feed_dict={x: 1}))

          



In [13]:
# sess.close()
